In [1]:
# imports
import pandas as pd
import numpy as np

# choropleth
import geopandas as gdp
import plotly.express as px
import us

### 1. combine and merge datasets

In [2]:
# spend data relative to january 6 to february 2, 2020 (% change from baseline)

spend = pd.read_csv('data/Affinity - State - Monthly.csv')

# add state names to spend df
states = pd.read_csv('data/GeoIDs - State.csv')

spend = spend.merge(states[['statefips', 'statename']], on='statefips')

# remove unnecessary columns
# features that start with 'spend_s' are not seasonally adjusted, remove them
drop = ['statefips', 'day_endofmonth', 'freq', 'provisional'] + [col for col in spend.columns if col.startswith('spend_s')]
spend = spend.drop(columns=drop)

# map col names to more descriptive ones
base_mapping = {
    "all": "all",
    "all_incmiddle": "all_incmiddle",
    "aap": "apparel and accessories",
    "acf": "accommodation and food services",
    "aer": "arts, entertainment, and recreation",
    "apg": "general merchandise + apparel",
    "gen": "general merchandise",
    "grf": "grocery and food stores",
    "hcs": "health care and social assistance",
    "hic": "home improvement centers",
    "sgh": "sporting goods and hobby",
    "tws": "transportation and warehousing",
    "retail_w_grocery": "retail including grocery",
    "retail_no_grocery": "retail excluding grocery",
    "durables": "durable goods",
    "nondurables": "non-durable goods",
    "remoteservices": "remote services",
    "inperson": "in-person services",
    "inpersonmisc": "other in-person services"
}

cols_with_quarters = [
    "spend_all_q1", "spend_all_q2", "spend_all_q3", "spend_all_q4",
    "spend_aap_q1", "spend_aap_q2", "spend_aap_q3", "spend_aap_q4",
    "spend_acf_q1", "spend_acf_q2" 
]

col_mapping = {}

for key, val in base_mapping.items():
    col_mapping[f"spend_{key}"] = 'spend ' + val

for col in cols_with_quarters:
    parts = col.split("_q")
    base = parts[0].replace("spend_", "")
    qnum = parts[1]
    if base in base_mapping:
        col_mapping[col] = f"spend {base_mapping[base]} q{qnum}"

spend = spend.rename(columns=col_mapping)

# convert all q1 to type float 
spend['spend all q1'] = [
    float(spend['spend all q1'][i]) if spend['spend all q1'][i] != '.' else np.nan
    for i in range(len(spend))
]

# group by state year
state_spend = spend.groupby(['statename', 'year', 'month']).sum().reset_index()
state_spend.head()

state_spend

,statename,year,month,spend apparel and accessories,spend accommodation and food services,"spend arts, entertainment, and recreation",spend all,spend general merchandise + apparel,spend durable goods,spend general merchandise,...,spend remote services,spend transportation and warehousing,spend all_incmiddle,spend all q1,spend all q2,spend all q3,spend all q4,spend in-person services,spend retail excluding grocery,spend retail including grocery
0,Alabama,2020,1,0.00000,-1.910000e-09,0.0000,0.0000,-2.880000e-08,3.280000e-08,0.0000,...,-6.630000e-09,0.0000,8.100000e-08,0.0000,0.0000,0.0000,0.0000,-3.680000e-08,-4.440000e-08,4.360000e-08
1,Alabama,2020,2,-0.04380,2.160000e-02,0.0643,0.0278,2.020000e-02,1.250000e-02,0.0782,...,4.500000e-02,-0.0412,8.730000e-03,0.0514,-0.0106,0.0348,0.0408,2.190000e-04,2.340000e-02,5.010000e-02
2,Alabama,2020,3,-0.35100,-3.800000e-01,-0.3730,-0.0898,-1.460000e-01,-3.850000e-02,0.0646,...,-2.870000e-02,-0.4600,-1.050000e-01,-0.0571,-0.1090,-0.0996,-0.1080,-3.280000e-01,-8.410000e-02,8.000000e-02
3,Alabama,2020,4,-0.53500,-6.060000e-01,-0.6520,-0.2290,-3.190000e-01,1.530000e-02,-0.0916,...,-1.210000e-01,-0.6440,-2.620000e-01,-0.1630,-0.2700,-0.2520,-0.2620,-5.840000e-01,-1.450000e-01,-3.380000e-02
4,Alabama,2020,5,-0.18200,-3.820000e-01,-0.4720,-0.1010,-7.270000e-02,1.720000e-01,0.0279,...,-5.080000e-02,-0.6160,-1.330000e-01,-0.0346,-0.1550,-0.1030,-0.1400,-3.680000e-01,1.160000e-01,1.290000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3514,Wyoming,2025,5,0.04100,2.570000e-01,0.1210,0.3640,7.310000e-01,3.330000e-01,1.3000,...,6.370000e-01,0.1660,3.860000e-01,0.4240,0.4800,0.3000,0.2810,-9.490000e-02,4.700000e-01,3.550000e-01
3515,Wyoming,2025,6,0.03540,2.330000e-01,0.2850,0.3570,7.770000e-01,2.620000e-01,1.4000,...,6.540000e-01,0.0567,3.690000e-01,0.4200,0.4650,0.2810,0.3120,-3.760000e-02,4.240000e-01,3.170000e-01
3516,Wyoming,2025,7,0.04590,2.700000e-01,0.6930,0.4110,8.850000e-01,3.200000e-01,1.6100,...,6.480000e-01,0.3090,4.240000e-01,0.4590,0.5010,0.3520,0.3610,-2.120000e-01,5.330000e-01,3.880000e-01
3517,Wyoming,2025,8,0.00191,2.530000e-01,0.4470,0.3600,8.030000e-01,2.170000e-01,1.5600,...,5.760000e-01,0.3400,3.550000e-01,0.4920,0.4110,0.3030,0.3380,-8.130000e-02,4.080000e-01,3.280000e-01


In [3]:
# employment data relative to january 4 to 31, 2020

employment = pd.read_csv('data/Employment - State - Weekly.csv')

# add state names to employment df
employment = employment.merge(states[['statefips', 'statename']], on='statefips')

# remove unnecessary columns
drop = ['statefips']

employment = employment.drop(columns=drop)

# map col names to more descriptive ones
col_mapping = {

    "emp": "all",
    
    # by income quartiles
    "emp_incq1": "q1",
    "emp_incq2": "q2",
    "emp_incq3": "q3",
    "emp_incq4": "q4",
    "emp_incmiddle": "inc middle",
    "emp_incbelowmed": "inc below_median",
    "emp_incabovemed": "inc above median",
    
    # by NAICS supersectors
    "emp_ss40": "trade_transport utilities",
    "emp_ss60": "professional business_services",
    "emp_ss65": "education health services",
    "emp_ss70": "leisure hospitality"
}

for key, val in col_mapping.items():
    employment = employment.rename(columns={key: 'emp ' + val})

# convert all emp columns to type float
for col in employment.columns:
    if col.startswith('emp '):
        employment[col] = [
            float(employment[col][i]) if employment[col][i] != '.' else np.nan
            for i in range(len(employment))
        ]
# sort employment df by state name, year, month, and day_endofweek
employment = employment.sort_values(by=['statename', 'year', 'month', 'day_endofweek'])

# group by state, year, month
state_emp = employment.groupby(['statename', 'year', 'month'], as_index=False).last()

state_emp.head()

,statename,year,month,day_endofweek,emp all,emp q1,emp q2,emp q3,emp q4,emp inc middle,emp inc below_median,emp inc above median,emp trade_transport utilities,emp professional business_services,emp education health services,emp leisure hospitality
0,Alabama,2020,1,31,-0.00474,-0.01700,0.00717,-0.00084,-0.00768,0.00311,-0.00574,-0.00343,-0.0355,-0.00263,0.0107,-0.0168
1,Alabama,2020,2,28,0.01510,0.00845,0.01100,0.02940,0.01040,0.02030,0.00962,0.02220,0.0163,-0.00342,0.0505,-0.0200
2,Alabama,2020,3,27,-0.09350,-0.19800,-0.07090,-0.03170,-0.03860,-0.05100,-0.13900,-0.03430,-0.1420,-0.05530,-0.0833,-0.3190
3,Alabama,2020,4,24,-0.21500,-0.41300,-0.18600,-0.10800,-0.07350,-0.14700,-0.30700,-0.09520,-0.2700,-0.17100,-0.2360,-0.5410
4,Alabama,2020,5,29,-0.17200,-0.30000,-0.13100,-0.13200,-0.06780,-0.13100,-0.22100,-0.10700,-0.2040,-0.28300,-0.1220,-0.2460


In [4]:
# covid-19 data

covid = pd.read_csv('data/COVID - State - Daily.csv')   

# add state names to covid df
covid = covid.merge(states[['statefips', 'statename']], on='statefips')

# remove uncessary columns and keep only rates
keep = ['year', 'month', 'statename'] + [col for col in covid.columns if col.endswith('_rate') and not col.startswith('new')]
covid = covid[keep]

# convert all rate columns to type float
for col in covid.columns:
    if col.endswith('_rate'):
        covid[col] = [
            float(covid[col][i]) if covid[col][i] != '.' else np.nan
            for i in range(len(covid))
        ]

# group by state, year, month
state_covid = covid.groupby(['statename', 'year', 'month']).mean().reset_index()
state_covid.head()

/var/folders/5r/fvb423zx5bgd_t4w2tvz41j40000gn/T/ipykernel_22064/1913648835.py:3: DtypeWarning: Columns (5,7,16,19,20,29) have mixed types. Specify dtype option on import or set low_memory=False.
  covid = pd.read_csv('data/COVID - State - Daily.csv')


,statename,year,month,case_rate,death_rate,test_rate,vaccine_rate,fullvaccine_rate,booster_first_rate,hospitalized_rate
0,Alabama,2020,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,2020,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,2020,3,3.848895,0.017158,NaN,NaN,NaN,NaN,0.101591
3,Alabama,2020,4,72.970000,2.257533,NaN,NaN,NaN,NaN,7.704000
4,Alabama,2020,5,224.387097,8.786129,NaN,NaN,NaN,NaN,8.569677


In [5]:
# mobility data relative to january 3 to february 6, 2020

mobility = pd.read_csv('data/Google Mobility - State - Daily.csv')

# add state names to mobility df
mobility = mobility.merge(states[['statefips', 'statename']], on='statefips')

# remove unnecessary columns
drop = ['statefips']
mobility = mobility.drop(columns=drop)

# map col names to more descriptive ones
col_mapping = {
    
}
# map col names to more descriptive ones
col_mapping = {

    'gps_retail_and_recreation': 'time retail and recreation',
    'gps_grocery_and_pharmacy': 'time grocery and pharmacy', 
    'gps_parks': 'time parks', 
    'gps_transit_stations': 'time transit stations',
    'gps_workplaces': 'time workplaces', 
    'gps_residential': 'time residential', 
    'gps_away_from_home': 'time away from home',
    
}

for key, val in col_mapping.items():
    mobility = mobility.rename(columns={key: val})

# convert mobility cols to type float
object_cols = ['time parks', 'time transit stations']

for col in object_cols:
    mobility[col] = [
        float(mobility[col][i]) if mobility[col][i] != '.' else np.nan
        for i in range(len(mobility))
        ]

mobility = mobility.sort_values(by=['statename', 'year', 'month', 'day'])

state_mobility = mobility.groupby(['statename', 'year', 'month'], as_index=False).last()
state_mobility.head()

,statename,year,month,day,time retail and recreation,time grocery and pharmacy,time parks,time transit stations,time workplaces,time residential,time away from home
0,Alabama,2020,2,29,0.0900,0.0414,0.191,0.1070,0.010,-0.00571,0.00855
1,Alabama,2020,3,31,-0.3340,-0.0643,0.180,-0.2340,-0.344,0.13100,-0.15900
2,Alabama,2020,4,30,-0.2400,-0.0300,0.143,-0.2300,-0.347,0.12100,-0.14800
3,Alabama,2020,5,31,-0.0914,0.0314,0.373,-0.0371,-0.236,0.08290,-0.09430
4,Alabama,2020,6,30,-0.0657,0.0171,0.429,0.0500,-0.234,0.07140,-0.08210


In [11]:
# merge all dfs by state, year, and month
df = state_spend.merge(state_emp, on=['statename', 'year', 'month'], how='left')
df = df.merge(state_covid, on=['statename', 'year', 'month'], how='left')
df = df.merge(state_mobility, on=['statename', 'year', 'month'], how='left')

# add recovered col for spend and emp
df.loc[df['spend all'].notna(), 'spend recovered'] = df.loc[df['spend all'].notna(), 'spend all'] >= 0
df.loc[df['emp all'].notna(), 'emp recovered'] = df.loc[df['emp all'].notna(), 'emp all'] >= 0

# convert to csv file
df.to_csv('data.csv', index=False)


In [12]:
df

,statename,year,month,spend apparel and accessories,spend accommodation and food services,"spend arts, entertainment, and recreation",spend all,spend general merchandise + apparel,spend durable goods,spend general merchandise,...,day,time retail and recreation,time grocery and pharmacy,time parks,time transit stations,time workplaces,time residential,time away from home,spend recovered,emp recovered
0,Alabama,2020,1,0.00000,-1.910000e-09,0.0000,0.0000,-2.880000e-08,3.280000e-08,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False
1,Alabama,2020,2,-0.04380,2.160000e-02,0.0643,0.0278,2.020000e-02,1.250000e-02,0.0782,...,29.0,0.0900,0.0414,0.191,0.1070,0.010,-0.00571,0.00855,True,True
2,Alabama,2020,3,-0.35100,-3.800000e-01,-0.3730,-0.0898,-1.460000e-01,-3.850000e-02,0.0646,...,31.0,-0.3340,-0.0643,0.180,-0.2340,-0.344,0.13100,-0.15900,False,False
3,Alabama,2020,4,-0.53500,-6.060000e-01,-0.6520,-0.2290,-3.190000e-01,1.530000e-02,-0.0916,...,30.0,-0.2400,-0.0300,0.143,-0.2300,-0.347,0.12100,-0.14800,False,False
4,Alabama,2020,5,-0.18200,-3.820000e-01,-0.4720,-0.1010,-7.270000e-02,1.720000e-01,0.0279,...,31.0,-0.0914,0.0314,0.373,-0.0371,-0.236,0.08290,-0.09430,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3514,Wyoming,2025,5,0.04100,2.570000e-01,0.1210,0.3640,7.310000e-01,3.330000e-01,1.3000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True
3515,Wyoming,2025,6,0.03540,2.330000e-01,0.2850,0.3570,7.770000e-01,2.620000e-01,1.4000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN
3516,Wyoming,2025,7,0.04590,2.700000e-01,0.6930,0.4110,8.850000e-01,3.200000e-01,1.6100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN
3517,Wyoming,2025,8,0.00191,2.530000e-01,0.4470,0.3600,8.030000e-01,2.170000e-01,1.5600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN


### 2. exploratory data analysis (eda)

In [18]:
# view spend recovery rates for each state every year after covid-19

state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT',
    'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN',
    'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND',
    'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC'
}

# select March of each year after 2020
df_jan = df[(df['month'] == 1) & (df['year'] > 2020)]
df_jan['year_label'] = 'Jan ' + df_jan['year'].astype(str)

# make a new col for state abbreviations
df_jan['state_abbrev'] = df_jan['statename'].map(state_abbrev)


fig = px.choropleth(
    df_jan,
    locations='state_abbrev',
    locationmode='USA-states',
    color='spend recovered',
    color_discrete_map={True: '#96b9d0', False: '#ffd5b6'},
    scope='usa',
    animation_frame='year_label', 
    title='Consumer Spending Recovery by State (Jan 2021–2025)',
    width=1000,
    height=600,
)


fig.show()


/var/folders/5r/fvb423zx5bgd_t4w2tvz41j40000gn/T/ipykernel_22064/915482598.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5r/fvb423zx5bgd_t4w2tvz41j40000gn/T/ipykernel_22064/915482598.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
# view emp recovery rates for each state every year after covid-19

fig = px.choropleth(
    df_jan,
    locations='state_abbrev',
    locationmode='USA-states',
    color='emp recovered',
    color_discrete_map={True: '#96b9d0', False: '#ffd5b6'},
    scope='usa',
    animation_frame='year_label', 
    title='Employment Rate Recovery by State (Jan 2021–2025)',
    width=1000,
    height=600,
)


fig.show()
